# Finetune Knowledge

In this tutorial we experiment with some basic approaches of "baking in knowledge with fine-tuning."

- Synthesizing questions from existing context
- Trying text completion

In [1]:
import os
import openai
from llama_index import ServiceContext
from llama_index.llms import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."
openai.api_key = os.environ["OPENAI_API_KEY"]

## Load Data

In [3]:
!mkdir data && wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

mkdir: data: File exists


In [2]:
from pathlib import Path
from llama_hub.file.pdf.base import PDFReader
from llama_hub.file.unstructured.base import UnstructuredReader
from llama_hub.file.pymu_pdf.base import PyMuPDFReader

In [3]:
# loader = PDFReader()
# docs0 = loader.load_data(file=Path("./data/llama2.pdf"))
# loader = UnstructuredReader()
# docs0 = loader.load_data(file=Path("./data/llama2.pdf"))

loader = PyMuPDFReader()
docs0 = loader.load(file_path=Path("./data/llama2.pdf"))


In [4]:
from llama_index import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

In [5]:
print(docs[0].get_content())

Llama 2: Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗
Louis Martin†
Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov

In [40]:
from llama_index.callbacks import CallbackManager, LlamaDebugHandler

# setup debug handler
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0.3),
    callback_manager=callback_manager
)
gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-0613", temperature=0.3),
    callback_manager=callback_manager
)


# scontext = gpt_35_context
scontext = gpt_4_context

## Generate Dataset

In [35]:
from llama_index.evaluation import DatasetGenerator
from llama_index.node_parser import SimpleNodeParser

In [8]:
node_parser = SimpleNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(docs)

In [9]:
print(nodes[2].get_content())

Figure 1: Helpfulness human evaluation results for Llama
2-Chat compared to other open-source and closed-source
models. Human raters compared model generations on ~4k
prompts consisting of both single and multi-turn prompts.
The 95% confidence intervals for this evaluation are between
1% and 2%. More details in Section 3.4.2. While reviewing
these results, it is important to note that human evaluations
can be noisy due to limitations of the prompt set, subjectivity
of the review guidelines, subjectivity of individual raters,
and the inherent difficulty of comparing generations.
Figure 2: Win-rate % for helpfulness and
safety between commercial-licensed base-
lines and Llama 2-Chat, according to GPT-
4. To complement the human evaluation, we
used a more capable model, not subject to
our own guidance. Green area indicates our
model is better according to GPT-4. To remove
ties, we used win/(win + loss). The orders in
which the model responses are presented to
GPT-4 are randomly swapped to

In [20]:
num_questions_per_chunk = 10
question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context, "
    f"formulate {num_questions_per_chunk} that captures an important fact from the "
    "context. \n"
    "- Restrict the question to the context information provided.\n"
    "- Do NOT create a question that cannot be answered from the context.\n"
    "- Phrase the question so that it can be asked generally (e.g. don't put \"given provided context\" in the question)" 
)

dataset_generator = DatasetGenerator(
    [nodes[2]],
    question_gen_query=question_gen_query,
    service_context=scontext,
)

In [21]:
# num_questions_per_chunk = 10
# question_gen_query = (
#     "You are a Teacher/ Professor. Your task is to setup "
#     "a quiz/examination. Using the provided context, "
#     f"formulate {num_questions_per_chunk} that captures an important fact from the "
#     "context. Restrict the question to the context information provided. "
#     "Do NOT create a question that cannot be answered from the context. "
# )

# dataset_generator = DatasetGenerator(
#     nodes,
#     question_gen_query=question_gen_query,
#     service_context=gpt_35_context,
# )

In [22]:
questions = dataset_generator.generate_questions_from_nodes(num=10)

In [23]:
# questions = ['What is the purpose of Large Language Models (LLMs) as mentioned in the context?',
#  'What are the two models developed and released in this work?',
#  'What are the two benchmarks on which Llama 2-Chat models were tested?',
#  'What is the maximum scale of parameters for Llama 2 and Llama 2-Chat models?',
#  'What are the limitations of human evaluations as mentioned in the context?',
#  'What is the training methodology of LLMs as described in the context?',
#  'What are the methods used to increase the safety of Llama 2 and Llama 2-Chat models?',
#  'How does the performance of Llama 2-Chat models compare to existing open-source models and closed-source models?',
#  'What are some of the novel observations made during the development of Llama 2 and Llama 2-Chat?',
#  'What are some of the closed "product" LLMs mentioned in the context?']

questions

['What is the purpose of Large Language Models (LLMs) as mentioned in the context?',
 'What are the two models developed and released in this work?',
 'What are the two benchmarks on which Llama 2-Chat models were tested?',
 'What is the maximum scale of parameters for Llama 2 and Llama 2-Chat models?',
 'What are the limitations of human evaluations as mentioned in the context?',
 'What is the training methodology of LLMs as described in the context?',
 'What are the methods used to increase the safety of Llama 2 and Llama 2-Chat models?',
 'How does the performance of Llama 2-Chat models compare to existing open-source models and closed-source models?',
 'What are some of the novel observations made during the development of Llama 2 and Llama 2-Chat?',
 'What are some of the closed "product" LLMs mentioned in the context?']

In [33]:
# try running a question through list index
from llama_index import SummaryIndex
index = SummaryIndex([nodes[2]], service_context=gpt_4_context)

In [28]:
# ask question
qindex = 8
query_engine = index.as_query_engine()
response = query_engine.query(questions[qindex])

In [29]:
print(response)

The context does not provide specific details about the novel observations made during the development of Llama 2 and Llama 2-Chat.


In [41]:
# try evaluation modules
from llama_index.evaluation import QueryResponseEvaluator, ResponseEvaluator
from llama_index import PromptTemplate

In [82]:
eval_tmpl = PromptTemplate(
    "Please tell if a given piece of information "
    "is supported by the context and reflects details in the context.\n"
    "You need to answer with either YES or NO.\n"
    "Answer YES if any of the context supports the information, even "
    "if most of the context is unrelated. "
    "Answer NO if the information itself is uninformative and doesn't contain specific aspects of the context. "
    "Some examples are provided below. \n\n"
    "Information: Apple pie is generally double-crusted.\n"
    "Context: An apple pie is a fruit pie in which the principal filling "
    "ingredient is apples. \n"
    "Apple pie is often served with whipped cream, ice cream "
    "('apple pie à la mode'), custard or cheddar cheese.\n"
    "It is generally double-crusted, with pastry both above "
    "and below the filling; the upper crust may be solid or "
    "latticed (woven of crosswise strips).\n"
    "Answer: YES\n"
    "Information: Apple pies tastes bad.\n"
    "Context: An apple pie is a fruit pie in which the principal filling "
    "ingredient is apples. \n"
    "Apple pie is often served with whipped cream, ice cream "
    "('apple pie à la mode'), custard or cheddar cheese.\n"
    "It is generally double-crusted, with pastry both above "
    "and below the filling; the upper crust may be solid or "
    "latticed (woven of crosswise strips).\n"
    "Answer: NO\n"
    "Information: I do not have any information about apples.\n"
    "Context: An apple pie is a fruit pie in which the principal filling "
    "ingredient is apples. \n"
    "Answer: NO\n"
    "Information: {query_str}\n"
    "Context: {context_str}\n"
    "Answer: "
)

# query_eval_tmpl = PromptTemplate(
#     "Your task is to evaluate if the response for the query \
#     is in line with the context information provided.\n"
#     "You have two options to answer. Either YES/ NO.\n"
#     "Answer - YES, if the response for the query \
#     is in line with context information otherwise NO.\n"
#     "Answer NO if the response doesn't actually answer the question (for instance, if it's missing context).\n"
#     "Query and Response: \n {query_str}\n"
#     "Context: \n {context_str}\n"
#     "Answer: "
# )

query_eval_tmpl = PromptTemplate(
    "Your task is to evaluate the following:\n"
    "1) If the response for the query isn't able to answer the question provided.\n"
    "2) If the response is hallucinated given the context.\n"
    "If either of these are true, answer NO.\n"
    "If both are not true, answer YES.\n"
    "
    
    "You be given 3 items: the query and response, as well as the context. Return YES or NO as the answer.\n"
    "Query and Response: \n {query_str}\n"
    "Context: \n {context_str}\n"
    "Answer: "
)

qr_eval = QueryResponseEvaluator(service_context=gpt_4_context, query_eval_prompt_tmpl=query_eval_tmpl)
r_eval = ResponseEvaluator(service_context=gpt_4_context, eval_prompt_tmpl=eval_tmpl)

In [83]:
eval1 = qr_eval.evaluate(questions[qindex], response)

**********
Trace: index_construction
    |_node_parsing ->  0.002664 seconds
      |_chunking ->  0.001917 seconds
**********
**********
Trace: query
    |_query ->  0.845491 seconds
      |_retrieve ->  0.000318 seconds
      |_synthesize ->  0.844921 seconds
        |_templating ->  5e-05 seconds
        |_llm ->  0.837866 seconds
**********


In [84]:
tmp_dict = llama_debug.get_llm_inputs_outputs()
print(len(tmp_dict))
print(tmp_dict[-1][0].payload["messages"][0].content)

10
Your task is to evaluate the following:
1) If the response for the query isn't able to answer the question provided.
2) If the response is hallucinated given the context.
If either of these are true, answer NO.
If both are not true, answer YES.
You be given 3 items: the query and response, as well as the context. Return YES or NO as the answer.
Query and Response: 
 Question: What are some of the novel observations made during the development of Llama 2 and Llama 2-Chat?
Response: The context does not provide specific details about the novel observations made during the development of Llama 2 and Llama 2-Chat.
Context: 
 Figure 1: Helpfulness human evaluation results for Llama
2-Chat compared to other open-source and closed-source
models. Human raters compared model generations on ~4k
prompts consisting of both single and multi-turn prompts.
The 95% confidence intervals for this evaluation are between
1% and 2%. More details in Section 3.4.2. While reviewing
these results, it is imp

In [85]:
eval1

'YES'

In [48]:
eval2 = r_eval.evaluate(response)

**********
Trace: index_construction
    |_node_parsing ->  0.002446 seconds
      |_chunking ->  0.00188 seconds
**********
**********
Trace: query
    |_query ->  0.322857 seconds
      |_retrieve ->  0.000327 seconds
      |_synthesize ->  0.322215 seconds
        |_templating ->  3e-05 seconds
        |_llm ->  0.316874 seconds
**********


In [56]:
tmp_dict = llama_debug.get_llm_inputs_outputs()
print(tmp_dict[0][0].payload["messages"][0].content)

Your task is to evaluate if the response for the query     is in line with the context information provided.
You have two options to answer. Either YES/ NO.
Answer - YES, if the response for the query     is in line with context information otherwise NO.
Answer NO if the response doesn't actually answer the question (for instance, if it's missing context).
Query and Response: 
 Question: What are some of the novel observations made during the development of Llama 2 and Llama 2-Chat?
Response: The context does not provide specific details about the novel observations made during the development of Llama 2 and Llama 2-Chat.
Context: 
 Figure 1: Helpfulness human evaluation results for Llama
2-Chat compared to other open-source and closed-source
models. Human raters compared model generations on ~4k
prompts consisting of both single and multi-turn prompts.
The 95% confidence intervals for this evaluation are between
1% and 2%. More details in Section 3.4.2. While reviewing
these results, 

In [ ]:
print((eval1, eval2))

In [40]:
print(nodes[0].text)

Llama 2 : Open Foundation and Fine-Tuned Chat Models
Hugo Touvron∗Louis Martin†Kevin Stone†
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov 